In [189]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [193]:
#read clean dataset
data = pd.read_pickle("data/MAD_BAR_all.pkl").reset_index(drop=True)

In [204]:
data.loc[301030, "meta"]

'{"Turista": {"Promo": {"price": 101.5, "seats": 248}, "Promo +": {"price": 107.95, "seats": 248}, "Flexible": {"price": 128.5, "seats": 248}}, "Preferente": {"Mesa": {"price": 346.4, "seats": 12}, "Promo": {"price": 99.6, "seats": 57}, "Promo +": {"price": 110.4, "seats": 57}, "Flexible": {"price": 216.5, "seats": 57}}, "Turista Plus": {"Promo": {"price": 95.6, "seats": 49}, "Promo +": {"price": 103.3, "seats": 49}, "Flexible": {"price": 154.2, "seats": 49}}}'

In [198]:
# generate train_id
data["train_id"] = data[["origin", "destination", "departure","arrival"]].sum(axis=1)

In [201]:
data["weekday"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").weekday()
)
data["depart_month"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").month
)
data["depart_hour"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").hour
)
# only jan, feb, march,
data = data.loc[data["depart_month"] < 4].reset_index(drop=True)

In [202]:
data["depart_year"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").year
)

In [203]:
data["depart_year"].value_counts()

2020    458876
Name: depart_year, dtype: int64

In [172]:
len(data["train_id"].unique())

1514

In [79]:
# a price-seat adatok a meta változóban vannak,
# ami egy mindig más struktúrájú dictionary,
# elég trükkös volt kinyerni, arra írtam ezt a fgv-t


def extract_meta(train):
    prices_seats = pd.DataFrame()
    for i in tqdm(range(train.shape[0])):
        dic_in = json.loads(train.meta[i])
        # extract values from tree dictionary
        while type(list(dic_in.values())[0]) != float:
            res = {key: list(value) for key, value in dic_in.items()}
            dic_out = {}
            for key in res.keys():
                for value in res[key]:
                    dic_out[key + "_" + value] = dic_in[key][value]
            dic_in = dic_out

        # check for seats, if no seat its the price
        for key in list(dic_in.keys()):
            if "seats" not in key:
                if "price" not in key:
                    dic_in[key + "_price"] = dic_in[key]
                    dic_in.pop(key)

        prices_seats = pd.concat(
            [prices_seats, pd.DataFrame.from_dict(dic_in, orient="index").T]
        ).reset_index(drop=True)
    return prices_seats

In [158]:
chunks = {
    i: pd.concat(
        [
            data.loc[i * 10000 : i * 10000 + 9999, :].reset_index(drop=True),
            extract_meta(
                data.loc[i * 10000 : i * 10000 + 9999, :].reset_index(drop=True)
            ),
        ],
        axis=1,
    )
    for i in range(45)
}

100%|██████████| 10000/10000 [00:39<00:00, 253.89it/s]


In [165]:
df = pd.concat(list(chunks.values()))

In [174]:
last_chunk = pd.concat(
    [
        data.loc[450000:, :].reset_index(drop=True),
        extract_meta(data.loc[450000:, :].reset_index(drop=True)),
    ]
)

100%|██████████| 8876/8876 [00:32<00:00, 274.96it/s]


In [175]:
df = pd.concat([df,last_chunk])

In [182]:
df = df.dropna(subset=["id"]).reset_index(drop=True)

In [185]:
df = df.drop(
    columns=[
        "Preferente_YOVOY_price",
        "Preferente_YOVOY_seats",
        "Turista Plus_YOVOY_price",
        "Turista Plus_YOVOY_seats",
        "Turista_YOVOY VERANO_price",
        "Turista_YOVOY VERANO_seats",
        "Preferente_YOVOY VERANO_price",
        "Preferente_YOVOY VERANO_seats",
        "Turista Plus_YOVOY VERANO_price",
        "Turista Plus_YOVOY VERANO_seats",
        "Preferente_Mesa_seats",
        "Turista_Mesa_seats",
        "Turista Plus_Mesa_seats",
    ]
)

In [187]:
df["days_till_dep"] = [
    (
        datetime.datetime.strptime(df.loc[i, "departure"][0:19], "%Y-%m-%d %H:%M:%S")
        - datetime.datetime.strptime(
            df.loc[i, "insert_date"][0:19], "%Y-%m-%d %H:%M:%S"
        )
    ).days
    for i in range(df.shape[0])
]

df["timedelta_till_dep"] = [
    (
        datetime.datetime.strptime(df.loc[i, "departure"][0:19], "%Y-%m-%d %H:%M:%S")
        - datetime.datetime.strptime(
            df.loc[i, "insert_date"][0:19], "%Y-%m-%d %H:%M:%S"
        )
    )
    for i in range(df.shape[0])
]

In [188]:
df.to_pickle("data/MAD_BAR_trains_v2.pkl")